In [14]:
import pickle
from collections import defaultdict
from utils.tools import dotdict
from utils.results_analysis import get_tuned_metrics, open_results
from utils.ipynb_helpers import read_data
import json
import os
import numpy as np
import yaml

In [15]:
bbtest_dirs = [f for f in os.listdir("bbtest_logs") if f.startswith("2023_02_21_06")]

with open("configs/stockformer/basic_PEMSBAY.yaml", "r") as file:
    args_base = dotdict(yaml.full_load(file))

full_test_dirs = []
full_test_dirs_args = []
bbtest_ids = []


for bbtest_id, dir in enumerate(_dir for _dir in bbtest_dirs if len(os.listdir(f"bbtest_logs/{_dir}"))==8):
    print(dir, len(os.listdir(f"bbtest_logs/{dir}")))
    # bbtest_dirs.append(f"bbtest_logs/{dir}")
    for vdir in os.listdir(f"bbtest_logs/{dir}"):
        if vdir.startswith("version"):
            # print(vdir)
            
            with open(f"bbtest_logs/{dir}/{vdir}/hparams.yaml", "r") as file:
                args = dotdict(yaml.load(file, Loader=yaml.FullLoader))
                full_test_dirs.append(f"bbtest_logs/{dir}/{vdir}")
                full_test_dirs_args.append(args)
                bbtest_ids.append(bbtest_id)
                
                
len(bbtest_ids)

2023_02_21_06_49_28_stockformer_sl16_ei325_dm512_nh8_el4_ebNone 8
2023_02_21_06_49_13_stockformer_sl16_ei325_dm512_nh4_el3_ebNone 8
2023_02_21_06_48_38_stockformer_sl16_ei325_dm512_nh8_el4_ebNone 8
2023_02_21_06_50_08_stockformer_sl64_ei325_dm512_nh8_el4_ebNone 8
2023_02_21_06_49_58_stockformer_sl32_ei325_dm512_nh8_el4_ebNone 8
2023_02_21_06_48_53_stockformer_sl16_ei325_dm512_nh8_el4_ebNone 8
2023_02_21_06_48_43_stockformer_sl16_ei325_dm512_nh8_el3_ebNone 8
2023_02_21_06_49_08_stockformer_sl16_ei325_dm512_nh4_el4_ebNone 8
2023_02_21_06_48_58_stockformer_sl16_ei325_dm256_nh8_el4_ebNone 8
2023_02_21_06_50_13_stockformer_sl128_ei325_dm512_nh8_el4_ebNone 8
2023_02_21_06_49_38_stockformer_sl16_ei325_dm512_nh8_el4_ebNone 8
2023_02_21_06_48_27_stockformer_sl16_ei325_dm512_nh8_el4_ebNone 8
2023_02_21_06_49_53_stockformer_sl16_ei325_dm512_nh8_el4_ebNone 8
2023_02_21_06_48_33_stockformer_sl16_ei325_dm512_nh8_el4_ebNone 8
2023_02_21_06_50_03_stockformer_sl32_ei325_dm512_nh8_el4_ebNone 8
2023_02_2

160

In [16]:
#  Open, Process, and Aggregate Test Data
bb_tpd_dict = defaultdict(
    lambda: {
        "train": {"trues": [], "preds": [], "dates": []},
        "val": {"trues": [], "preds": [], "dates": []},
        "test": {"trues": [], "preds": [], "dates": []},
    }
)


df = read_data(os.path.join(args_base.root_path, args_base.data_path))
exceptions = []
for bbtest_id, log_dir, args in zip(bbtest_ids, full_test_dirs, full_test_dirs_args):
    args = dotdict(args)

    for data_group in ["train", "val", "test"]:
        tpd_dict = open_results(log_dir, args, df)

        true = tpd_dict[data_group]["trues"]
        pred = tpd_dict[data_group]["preds"]
        date = tpd_dict[data_group]["dates"]
        bb_tpd_dict[bbtest_id][data_group]["trues"].append(true)
        bb_tpd_dict[bbtest_id][data_group]["preds"].append(pred)
        bb_tpd_dict[bbtest_id][data_group]["dates"].append(date)

# Aggregate and cast
for bbtest_id in bb_tpd_dict.keys():
    for data_group in ["train", "val", "test"]:
        bb_tpd_dict[bbtest_id][data_group]["trues"] = np.concatenate(
            bb_tpd_dict[bbtest_id][data_group]["trues"]
        )
        bb_tpd_dict[bbtest_id][data_group]["preds"] = np.concatenate(
            bb_tpd_dict[bbtest_id][data_group]["preds"]
        )
        bb_tpd_dict[bbtest_id][data_group]["dates"] = bb_tpd_dict[bbtest_id][
            data_group
        ]["dates"][0].union_many(bb_tpd_dict[bbtest_id][data_group]["dates"][1:])

    with open(
        os.path.join("bbtest_logs", bbtest_dirs[bbtest_id], "tpd_dict.pickle"), "wb"
    ) as handle:
        pickle.dump(
            bb_tpd_dict[bbtest_id], handle, protocol=pickle.HIGHEST_PROTOCOL
        )

OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDING trues with df target
OVERRIDI

/tmp/ipykernel_154009/3196264813.py:37: FutureWarning: DatetimeIndex.union_many is deprecated and will be removed in a future version. Use obj.union instead.
  ]["dates"][0].union_many(bb_tpd_dict[bbtest_id][data_group]["dates"][1:])


In [17]:
#### Analyze
for bbtest_id in bb_tpd_dict.keys():
    best_thresh, best_thresh_metrics, zero_thresh_metrics = get_tuned_metrics(
        args, bb_tpd_dict[bbtest_id]
    )
    metrics = {0.0: zero_thresh_metrics, best_thresh: best_thresh_metrics}
    with open(os.path.join("bbtest_logs", bbtest_dirs[bbtest_id], "metrics.json"), "w") as f:
        json.dump(metrics, f, indent=2)

    # Warnings
    action_diff = np.abs(
        metrics[0.0]["test"]["pct_excluded_nshort"]
        - metrics[0.0]["test"]["pct_excluded_oshort"]
    )
    if action_diff > 0.6:
        print("WARNING: significant action preference between buying shorting")

    train_pct_dir_correct = metrics[0.0]["train"]["pct_dir_correct"]
    if train_pct_dir_correct < 0.55:
        print(
            f"WARNING: train isn't properly learning direction. pct_dir_correct: {train_pct_dir_correct}"
        )

    print("bbtest logged in:", bbtest_dirs[bbtest_id])

50'th percentile: 4.190291404724121


/home/zacschulwolf/Stockformer2022/utils/stock_metrics.py:119: RuntimeWarning: overflow encountered in exp
  return np.exp(apply_short_filter(output, raw, short_filter).sum())
/home/zacschulwolf/Stockformer2022/utils/stock_metrics.py:168: RuntimeWarning: overflow encountered in exp
  return np.exp(apply_short_filter(output, raw, short_filter).sum())
/home/zacschulwolf/Stockformer2022/utils/stock_metrics.py:206: RuntimeWarning: invalid value encountered in divide
  mult_min = np.log(-pct_change_mult + 2) / log_pct_change
/home/zacschulwolf/Stockformer2022/utils/stock_metrics.py:215: RuntimeWarning: overflow encountered in exp
  return np.exp(apply_short_filter(output, raw, short_filter).sum())


zeros thresh:
train	{  'avg_pct_profit_always_buy': 0.9999998649866929,
   'avg_pct_profit_always_short': 1.0000001350133254,
   'avg_pct_profit_tanhv1': inf,
   'pct_dir_correct': 0.5826681919798798,
   'pct_excluded': 0.0,
   'pct_excluded_nshort': 0.5205131339946905,
   'pct_excluded_oshort': 0.4794868660053095,
   'pct_profit_dir': inf,
   'pct_profit_dir_nshort': inf,
   'pct_profit_dir_opt': inf,
   'pct_profit_dir_oshort': inf,
   'pct_profit_tanhv1': inf,
   'pct_profit_tanhv1_nshort': inf,
   'pct_profit_tanhv1_oshort': inf,
   'pct_profit_tanhv2': inf,
   'pct_profit_tanhv2_nshort': inf,
   'pct_profit_tanhv2_oshort': inf}
val	{  'avg_pct_profit_always_buy': 0.9999993617059114,
   'avg_pct_profit_always_short': 1.000000638294496,
   'avg_pct_profit_tanhv1': 1.0048314201792248,
   'pct_dir_correct': 0.5302269345238095,
   'pct_excluded': 0.0,
   'pct_excluded_nshort': 0.5018807870370371,
   'pct_excluded_oshort': 0.49811921296296297,
   'pct_profit_dir': 8.142493138607956e+205